# Proyecto 4: Faces in The Wild

**Integrantes**:
- Lucas Carranza
- David Herencia
- Kalos Lazo
- Lenin Chavez

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

using device: cuda:0


## Exploración del Dataset

In [3]:
import os, glob

In [4]:
def plot_imgs(imgs, size=3):
    img_number = imgs.shape[0]
    rows = cols = math.ceil(np.sqrt(img_number))
    fig = plt.figure(figsize=(rows*size, cols*size))
    for i in range(img_number):
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(imgs[i].detach(), cmap='gray')
        plt.axis('off')
    plt.show()

## Modelo 1: CNN

## Modelo 2: ??

## Experimentación

## Testing